```{currentmodule} optimap
```

In [ ]:
# Code snippet for rendering animations in the docs
from IPython.display import HTML
import warnings
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
import matplotlib.pyplot as plt

def render_ani_func(f):
    om.utils.disable_interactive_backend_switching()
    plt.switch_backend('Agg')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ani = f()
    %matplotlib inline
    om.utils.enable_interactive_backend_switching()

    vid = HTML(ani.to_html5_video())
    plt.close('all')
    return vid

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/signal_extraction.ipynb>`, or a {download}`python script <converted/signal_extraction.py>` with code cells.
```

# Tutorial 2: Measurement of Fluorescent Signals and Waves

This tutorial will cover the measurement of fluorescent signals and wave dynamics from cardiac optical mapping data more generally. In [Tutorial 1](basics.ipynb) we already loaded and displayed video data, performed numerical motion tracking and motion-stabilization, performed some post-processing, and displayed action potential waves. In this tutorial, we will cover the post-processing routines used to extract and display the optical signals in more detail. We first repeat the steps from the previous tutorial by loading the experimental data and performing motion correction. Note that the motion correction step is not required if you analyze data that was acquired with Blebbistatin or other pharmacological uncoupling or motion-inhibition techniques (you then simply skip this step).

In [ ]:
import optimap as om
import numpy as np

filepath = om.utils.retrieve_example_data('Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um.npy')
# alterantively set filepath = '/folder_on_your_computer/Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um.npy'
video = om.load_video(filepath)
video_warped = om.motion_compensate(video,
                              contrast_kernel=5,
                              presmooth_spatial=1,
                              presmooth_temporal=1)
om.video.play(video_warped, title="recording without motion", skip_frame=3);

In [ ]:
# Hidden, same as above but shorten the video slightly

import optimap as om
import numpy as np

filepath = om.utils.retrieve_example_data('Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um.npy')
video = om.load_video(filepath, use_mmap=True)[:500]
filepath = om.utils.retrieve_example_data('Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um_warped.npy', silent=True)
video_warped = om.load_video(filepath, use_mmap=True)[:500]
render_ani_func(lambda: om.video.play(video_warped, title="recording without motion", skip_frame=1, interval=20))

Measuring action potentials or calcium transients in cardiac optical mapping videos is conventionally performed pixel-by-pixel. In the absence of motion, either with Blebbistatin or after numerical motion inhibition, each pixel shows the same tissue segment throughout the video. This condition is a prerequisite for the following processing routines.

## Extraction of Fluorescenct Signals and Waves from raw Video Data

Voltage- or calcium-sensitive dyes or indicators modulate the intensity of the tissue in response to changes in the transmembrane potential or intracellular calcium concentration, respectively. When we extract and plot time-series from a video which show the pixel's intensities over time we will see small fluctuations or optical signals which correspond to action potentials or calcium transients: 

In [ ]:
positions = [(192, 167), (204, 141), (118, 158), (183, 267)]
traces = om.extract_traces(video_warped, positions, size=3, show=True, fps=500)

You will notice that the optical signals are small compared to the overall background fluorescence ($\Delta F$ vs. $F$). Moreover, each time-series has its own particular baseline, which correlates with the brightness in the video image. In this example, the background is in the order of 2000-4000 intensity counts, and the small signal fluctuations are in the order of 100-200 intensity counts (the data was generated with a Basler acA720-520um camera with a 12-bit dynamic range). This signal needs to be isolated from the raw data. In other terms, the background fluorescence needs to be removed and the remaining signal needs to be amplified or "renormalized" before it can be further processed or visualized. This background-removal and amplification process can be done using 3 different approaches:

1. Using pixel-wise normalization: This method conserves the shape of the action potential (under ideal conditions) and normalizes the signal to [0, 1].
2. Using sliding-window pixel-wise normalization: This method is a variant of method 1 which also conserves the shape of the action potential (under ideal conditions and with restrictions) and normalizes the signal to [0, 1] using a sliding window. However, it can be sensitive to noise and changes of the base fluorescence due to inhomogeneous illumination when using motion correction.
3. Computing the frame (or temporal) difference: This method is more robust than methods 1 and 2, however, it does not conserve the shape of the action potential as it amplifies rapid temporal changes in the signal. It is useful to visualize the propagation of the front of the action potential waves.

### 1. Pixel-wise Normalization

Pixel-wise normalization refers to renormalizing each time-series or optical trace obtained from a single pixel individually. Renormalizing means that first the minimum of the time-series is subtracted from all values in the time-series, which removes the baseline or background fluorescence, and then all values are divided by the range (maximum value - minimum value) of the time-series, which produces a signal that fluctuates between 0 and 1. In ideal conditions, inactivated tissue then corresponds to 0 and the peaks of action potentials or calcium transients correspond to 1. A pixel-wise normalized signal can be computed for each pixel $(x, y)$ and frame $t$ using the following equation:

$$
\begin{align}
    \text{signal}_{\text{norm}}(t, x, y) = \frac{\text{signal}(t, x, y) - \text{min}_{t} \text{signal}(t, x, y)}{\text{max}_{t} \text{signal}(t, x, y) - \text{min}_{t} \text{signal}(t, x, y)}
\end{align}
$$

The function {func}`video.normalize_pixelwise` performs this operation automatically and produces a pixel-wise normalized video:

In [ ]:
video_warped_pnorm = om.video.normalize_pixelwise(video_warped)

The pixel-wise normalized video looks as follows:

In [ ]:
om.video.play(video_warped_pnorm, title="sliding window normalization", interval=20);

In [ ]:
render_ani_func(lambda: om.video.play(video_warped_pnorm, title="sliding window normalization", interval=20))

You may have noticed that the video gets darker over time. The pixel-wise normalization does not account for overall brightness changes and baseline drifts as only the maximal and minimal value in the entire time-series are considered.

The optical traces sampled from the pixel-wise normalized video at the same locations as above look as follows:

In [ ]:
traces_pnorm = om.extract_traces(video_warped_pnorm, positions, size=3, show=True, fps=500)
traces_pnorm = om.extract_traces(video_warped_pnorm, positions[0], size=3, show=True, fps=500)

The different baselines were removed and all signals now lie within the interval [0,1]. However, you can notice undesired baseline drifts in the optical traces which cannot be removed using the pixel-wise normalization. Moreover, due to the noise, the normalization does not scale the signal all the way to the interval [0,1] as the minima and maxima are likely outliers associated with the noise. The video data would have to be smoothed to obtain better results.

### 2. Sliding-Window Pixel-wise Normalization

The sliding-window (or rolling-window )pixel-wise normalization performs a pixel-by-pixel normalization of the signal to [0, 1] within a short temporal window for each time point. For each point in time, a new minimum and maximum is calculated within a temporal window around the point. The function {func}`video.normalize_pixelwise_slidingwindow` performs this operation by computing the following equation for each pixel $(x, y)$ and frame $t$:

$$
\begin{align}
    \text{signal}_{\text{norm}}(t, x, y) = \frac{\text{signal}(t, x, y) - \text{min}_{t' \in [t - w/2, t + w/2]} \text{signal}(t', x, y)}{\text{max}_{t' \in [t - w/2, t + w/2]} \text{signal}(t', x, y) - \text{min}_{t' \in [t - w/2, t + w/2]} \text{signal}(t', x, y)}
\end{align}
$$

where $w$ is the window size. The window size $w$ needs to be adjusted to the period of the action potential or calcium wave activity and should not be shorter than the period and ideally be roughly 1-3 periods long. In contrast to the pixel-wise normalization, the sliding-window pixel-wise normalization can inhibit baseline drifts. If the window is too long, baseline drifts will not be inhibited as effectively as with shorter windows. We will use a window size $w$ of 60 frames (corresponds to 120 ms) for this example:


In [ ]:
video_warped_spnorm = om.video.normalize_pixelwise_slidingwindow(video_warped, window_size=60)

The sliding-window pixel-wise normalized video looks as follows:

In [ ]:
om.video.play(video_warped_spnorm, title="sliding window normalization", interval=20);

In [ ]:
render_ani_func(lambda: om.video.play(video_warped_spnorm, title="sliding window normalization", interval=20))

Unlike the pixel-wise normalized video, this video does not get darker towards the end. Accordingly, the optical traces show minimal baseline modulations:

In [ ]:
traces_spnorm = om.extract_traces(video_warped_spnorm, positions, size=3, show=True, fps=500)
traces_spnorm = om.extract_traces(video_warped_spnorm, positions[0], size=3, show=True, fps=500)


```{warning}
This tutorial is currently work in progress. We will add more information soon.
```

In order to blank-out everything else other than tissue, we can use ``optimap``'s mask function:

In [ ]:
background_mask = om.background_mask(video_warped[0])

In [ ]:
video_warped_norm = om.video.normalize_pixelwise_slidingwindow(video_warped, window_size=60)
video_warped_norm[:, background_mask] = np.nan

om.video.play(video_warped_norm, title="sliding window normalization", interval=20);

In [ ]:
video_warped_norm = om.video.normalize_pixelwise_slidingwindow(video_warped, window_size=60)
video_warped_norm[:, background_mask] = np.nan

render_ani_func(lambda: om.video.play(video_warped_norm, title="sliding window normalization", interval=20))

For better visualization in the next steps we'll need a mask of the background. Here we use {func}`background_mask` to create a mask of the background from the first frame of the data using an automatic threshold.

Pixels in red are considered background (value of `True`), and pixels in blue are considered foreground (value of `False`).

In [ ]:
alpha = om.video.normalize(video_warped_norm, vmin=0.5, vmax=0)
om.video.play_with_overlay(video_warped, 1-video_warped_norm, alpha=alpha, vmin_overlay=0, vmax_overlay=1, interval=30);

In [ ]:
alpha = om.video.normalize(video_warped_norm, vmin=0.5, vmax=0)
render_ani_func(lambda: om.video.play_with_overlay(video_warped, (1-video_warped_norm), alpha=alpha, interval=20))

### 3. Frame/Temporal Difference

The temporal difference approach computes the difference between frames at time $t$ and $t - n$:

$$
\begin{align}
    \text{signal}_{\text{diff}}(t, x, y) = \text{signal}(t, x, y) - \text{signal}(t - n, x, y)
\end{align}
$$

where $\Delta t$ is the time difference between frames. This method is more robust, however it does generally not conserve the shape of the action potential. Rather, it isolates the front of the fluorescent waves.

Let's use $n = 10$ for this example.

In [ ]:
n = 5
diff = om.video.temporal_difference(video_warped, n)
diff[:, background_mask] = np.nan
abs_max = 0.33*np.nanmax(np.abs(diff))
om.video.play(diff, title="temporal difference", cmap="PiYG", vmin=-abs_max, vmax=abs_max, interval=20);

In [ ]:
diff[diff > 0] = 0
om.video.play(diff, title="temporal difference", cmap="PiYG", vmin=-abs_max, vmax=abs_max, interval=20);

In [ ]:
om.video.play2(video_warped_norm, diff, title1="Sliding Window", title2="Frame Difference", vmin1=0, vmax1=1, cmap2='gray', vmin2=0.2*np.nanmin(diff), vmax2=0, interval=20);

In [ ]:
diff_norm = om.video.normalize_pixelwise_slidingwindow(diff, window_size=60)
om.video.play2(video_warped_norm, diff_norm, vmin1=0, vmax1=1, cmap2='gray', vmin2=0, vmax2=1, interval=20);

In [ ]:
diff = om.video.temporal_difference(video_warped, 10)
diff[diff > 0] = 0
diff[:, background_mask] = np.nan
diff_norm = om.video.normalize_pixelwise_slidingwindow(-diff, window_size=60)
om.video.play_with_overlay(video_warped_norm, diff_norm, vmin_overlay=-1, vmax_overlay=1)

In [ ]:
diff = om.video.temporal_difference(video_warped, 10)
diff[diff > 0] = 0
diff[:, background_mask] = np.nan
diff_norm = om.video.normalize_pixelwise_slidingwindow(-diff, window_size=60)
render_ani_func(lambda: om.video.play_with_overlay(video_warped, diff_norm, vmin_overlay=-1, vmax_overlay=1, interval=20))